In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

tfx version 1.16.0 is latest stable as of Sep 28, 2025

It is compatible with python 3.9 and 3.10 only.

The current kaggle python docker image uses python 3.11.13.

To use an earlier version of python on Kaggle, one can install conda and create a virtual environment that is based on an earlier version of python. 

Once conda is installed and a virtual environment is created for the earlier version of python, the virtual environment can be activated by activating conda and then activating the virtual environment.

A bash shell in the notebook that is invoked from the magic command %%bash is a bash session for the extent of that specific cell.
For each new session invoked by the cell %%bash, the 2 activation commands need to be invoked before using the virtual environment.

Aside from running scripts in the magic bash shell cells, we can also run scripts using the python subprocess library as long as we prepend commands with the 2 conda activation statements (see details in the definition for the run_command below).

We have 2 ways to run commands within the virtual environment.

The notebook itself is still using the kaggle docker image environment without the newly built virtual environment.
Even if we install and use ipykernel to register a kernel for the new virtual environment, I don't see a way to open the notebook to use the new kernel.

So, the notebook can be used for intermediate steps of EDA that 
use libraries that don't require an earlier version of python, else the virtual environment for anything needing earlier python version.


In [2]:
!pwd
!echo $HOME

/kaggle/working
/root


%%bash: Executes the entire cell as a shell script. 

In [3]:
%%bash
t0=$(date +%s%N)
mkdir -p ~/miniconda3
wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
#install conda and activate to /usr/local
bash ~/miniconda3/miniconda.sh -b -u -p /usr/local
rm ~/miniconda3/miniconda.sh
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r

. /usr/local/bin/activate
echo "**$SHELL**"
echo "**$BASH**"
conda init --all

. /root/.bashrc
conda create -q --name my_tfx_env python=3.10 -y
conda activate my_tfx_env
python --version

t1=$(date +%s%N)
t2=$(echo "scale=9;($t1-$t0) / 1000000000" | bc)
echo $t2 seconds
date

PREFIX=/usr/local
Unpacking bootstrapper...
Unpacking payload...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories of packages that are compatible with the Python interpreter
    in Miniconda3: /usr/local
accepted Terms of Service for https://repo.anaconda.com/pkgs/main
accepted Terms of Service for https://repo.anaconda.com/pkgs/r
**/bin/bash**
**/usr/bin/bash**
no change     /usr/local/condabin/conda
no change     /usr/local/bin/conda
no change     /usr/local/bin/conda-env
no change     /usr/local/bin/activate
no change     /usr/local/bin/deactivate
no change     /usr/local/etc/profile.d/conda.sh
no change     /usr/local/etc/fish/conf.d/conda.fish
no change   

to activate the conda environment, need to source from conda's activate (which I installed in /usr/local/bin above), then activate the conda virtual environment.

this has to be done for each magic shell cell

In [4]:
%%bash
t0=$(date +%s%N)
. /usr/local/bin/activate
conda activate my_tfx_env
python --version

#consider conda install ipykernel
conda install pip

#see dependencies https://github.com/tensorflow/transform
pip -q install pyarrow==10.0.1
pip -q install apache-beam==2.59.0
pip -q install tensorflow==2.16.1
pip -q install tensorflow-transform==1.16.0
pip -q install tfx==1.16.0
#
#tf metadata 1.16.1
#tfx-bsl
#keeps protobuf 3.20.3
#if use sparkrunner, install pyspark 4.0.0 or 3.3.x

pip list
t1=$(date +%s%N)
t2=$(echo "scale=9;($t1-$t0) / 60000000000" | bc)
echo "$t2 minutes"
date
#about 6-7 minutes for this cell.

Python 3.10.18
2 channel Terms of Service accepted
Channels:
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.

Package                            Version
---------------------------------- --------------
absl-py                            1.4.0
annotated-types                    0.7.0
anyio                              4.11.0
apache-beam                        2.59.0
argon2-cffi                        25.1.0
argon2-cffi-bindings               25.1.0
arrow                              1.3.0
astunparse                         1.6.3
async-lru                          2.0.5
async-timeout                      5.0.1
attrs                              23.2.0
babel                              2.17.0
backcall                           0.2.0
beautifulsoup4                     4.14.2
bleach                             6.2.0
cachetools                         5.5.2
certifi                            2025.8.3
cffi                               2



==> WARNING: A newer version of conda exists. <==
    current version: 25.7.0
    latest version: 25.9.0

Please update conda by running

    $ conda update -n base -c defaults conda


  DEPRECATION: Building 'crcmod' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'crcmod'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'dill' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to

In [5]:
%%bash
. /usr/local/bin/activate
conda activate my_tfx_env
python --version

Python 3.10.18


The run_command is from
https://www.kaggle.com/code/taylorsamarel/change-python-version-kaggle-v2-taylor-amarel

In [6]:
import subprocess
def run_command(cmd, capture=True, check=False):
    cmds = f". /usr/local/bin/activate; conda activate my_tfx_env; {cmd}"
    try:
        result = subprocess.run(cmds, shell=True, capture_output=capture, text=True, check=check)
        if capture:
            return result.stdout.strip() if result.stdout else result.stderr.strip()
        return result.returncode == 0
    except Exception as e:
        return str(e)

In [7]:
print(run_command("python --version"))

Python 3.10.18


In [8]:
%%bash
wget -q http://files.grouplens.org/datasets/movielens/ml-1m.zip -O /kaggle/working/ml-1m.zip
unzip -o /kaggle/working/ml-1m.zip
ls /kaggle/working/ml-1m/
rm /kaggle/working/ml-1m.zip

Archive:  /kaggle/working/ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
movies.dat
ratings.dat
README
users.dat


In [49]:
%%bash
#it can take a couple of minutes to get current version of recently uploaded file to github
#wget -q -c --no-cache https://raw.githubusercontent.com/nking/recommender_systems/refs/heads/main/src/test/python/test_tft.py -O /kaggle/working/test_tft.py
#curl --header "Cache-Control: no-cache" "https://api.github.com/repos/nking/recommender_systems/content/src/test/python/test_tft.py" -o /kaggle/working/test_tft.py

rm -f /kaggle/working/dataset_tfxio_example.py
wget -q -c --no-cache https://raw.githubusercontent.com/nking/recommender_systems/refs/heads/main/src/test/python/dataset_tfxio_example.py -O /kaggle/working/dataset_tfxio_example.py
#ls -l /kaggle/working/

repo_uri='https://raw.githubusercontent.com/nking/recommender_systems/refs/heads/development/src/main/python'
declare -a my_files=("ingest_movie_lens_beam.py" "CustomUTF8Coder.py" "ingest_movie_lens_component.py" "partition_funcs.py" "stringify_ingest_params.py")
for item in "${my_files[@]}"
do
  rm -f "/kaggle/working/$item"
  echo "$item"
  wget -q -c --no-cache "$repo_uri/$item" -O /kaggle/working/$item
done

repo_uri='https://raw.githubusercontent.com/nking/recommender_systems/refs/heads/development/src/drafts/python'
declare -a my_files=("ingest_movie_lens_tfx.py")
for item in "${my_files[@]}"
do
  rm -f "/kaggle/working/$item"
  echo "$item"
  wget -q -c --no-cache "$repo_uri/$item" -O /kaggle/working/$item
done

#repo_uri='https://raw.githubusercontent.com/nking/recommender_systems/refs/heads/development/src/test/python'
#declare -a my_files=("IngestMovieLensComponentTest.py")
#for item in "${my_files[@]}"
#do
#  rm -f "/kaggle/working/$item"
#  echo "$item"
#  wget -q -c --no-cache "$repo_uri/$item" -O /kaggle/working/$item
#done

ls -l /kaggle/working/
date

ingest_movie_lens_beam.py
CustomUTF8Coder.py
ingest_movie_lens_component.py
partition_funcs.py
stringify_ingest_params.py
ingest_movie_lens_tfx.py
total 80
drwxr-xr-x 4 root root  4096 Oct  2 23:22 bin
-rw-r--r-- 1 root root   756 Oct  3 00:13 CustomUTF8Coder.py
-rw-r--r-- 1 root root  2399 Oct  3 00:13 dataset_tfxio_example.py
-rw-r--r-- 1 root root 11011 Oct  3 00:13 ingest_movie_lens_beam.py
-rw-r--r-- 1 root root  8836 Oct  3 00:13 ingest_movie_lens_component.py
-rw-r--r-- 1 root root  4112 Oct  2 22:59 IngestMovieLensComponentTest.py
-rw-r--r-- 1 root root 18824 Oct  3 00:13 ingest_movie_lens_tfx.py
drwxr-x--- 2 root root  4096 Jan 29  2016 ml-1m
-rw-r--r-- 1 root root  2707 Oct  3 00:13 partition_funcs.py
drwxr-xr-x 2 root root  4096 Oct  3 00:06 __pycache__
-rw-r--r-- 1 root root  2206 Oct  3 00:13 stringify_ingest_params.py
Fri Oct  3 12:13:47 AM UTC 2025


In [10]:
#test that TFX and associated libraries are compatible:
print(run_command("python3 /kaggle/working/dataset_tfxio_example.py"))

{'x_centered': [[-4.0], [-3.0], [-2.0], [-1.0], [0.0]],
 'x_scaled': [[0.0], [0.125], [0.25], [0.375], [0.5]]}
{'x_centered': [[1.0], [2.0], [3.0], [4.0]],
 'x_scaled': [[0.625], [0.75], [0.875], [1.0]]}


In [50]:
%%bash

. /usr/local/bin/activate
conda activate my_tfx_env

python --version

echo "run ingest using beam and TFX for python custom component"

t0=$(date +%s%N)

python /kaggle/working/ingest_movie_lens_component.py

t1=$(date +%s%N)
t2=$(echo "scale=9;($t1-$t0) / 1000000000" | bc)
echo $t2 seconds
date

Python 3.10.18
run ingest using beam and TFX for python custom component
TensorFlow version: 2.16.1
TFX version: 1.16.0
begin test
tests done
62.280741259 seconds
Fri Oct  3 12:14:58 AM UTC 2025


INFO:absl:Using deployment config:
 executor_specs {
  key: "ingest_movie_lens_component"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "__main__.ingest_movie_lens_component_Executor"
      }
    }
  }
}
metadata_connection_config {
  database_connection_config {
    sqlite {
      filename_uri: "/kaggle/working/bin/metadata/movie_ens_ingest_test/metadata.db"
      connection_mode: READWRITE_OPENCREATE
    }
  }
}

INFO:absl:Using connection config:
 sqlite {
  filename_uri: "/kaggle/working/bin/metadata/movie_ens_ingest_test/metadata.db"
  connection_mode: READWRITE_OPENCREATE
}

INFO:absl:Component ingest_movie_lens_component is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "__main__.ingest_movie_lens_component"
  }
  id: "ingest_movie_lens_component"
}
contexts {
  contexts {
    type {
      name: "pipeline"
    }
    name {
      field_value {
        string_value: "movie_ens_ingest_test"
      }
    }
  }
  contexts

In [51]:
%%bash

. /usr/local/bin/activate
conda activate my_tfx_env

python --version

echo "run ingest using beam and TFX for fully custom component"

t0=$(date +%s%N)

python /kaggle/working/ingest_movie_lens_tfx.py

t1=$(date +%s%N)
t2=$(echo "scale=9;($t1-$t0) / 1000000000" | bc)
echo $t2 seconds
date

Python 3.10.18
run ingest using beam and TFX for fully custom component
TensorFlow version: 2.16.1
TFX version: 1.16.0
begin test
executor_spec=<tfx.dsl.components.base.executor_spec.ExecutorClassSpec object at 0x78cd2980f970>
DEBUG IngestMovieLensComponent init
11.453704324 seconds
Fri Oct  3 12:15:41 AM UTC 2025


INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Using deployment config:
 executor_specs {
  key: "IngestMovieLensComponent"
  value {
    beam_executable_spec {
      python_executor_spec {
        class_path: "__main__.IngestMovieLensExecutor"
      }
    }
  }
}
metadata_connection_config {
  database_connection_config {
    sqlite {
      filename_uri: "/kaggle/working/bin/metadata/MovieLensIngestTest/metadata.db"
      connection_mode: READWRITE_OPENCREATE
    }
  }
}

INFO:absl:Using connection config:
 sqlite {
  filename_uri: "/kaggle/working/bin/metadata/MovieLensIngestTest/metadata.db"
  connection_mode: READWRITE_OPENCREATE
}

INFO:absl:Component IngestMovieLensComponent is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "__main__.IngestMovieLensComponent"
  }
  id: "IngestMovieLensComponent"
}
contexts {
  contexts {
    type {
      name: "pipeline"
    }
    name {
      field_value {
        string_value: "MovieLensIngest

In [12]:
%%bash

. /usr/local/bin/activate
conda activate my_tfx_env

python --version

echo "run ingest using beam and DirectRunner"

t0=$(date +%s%N)

python /kaggle/working/ingest_movie_lens_beam.py

t1=$(date +%s%N)
t2=$(echo "scale=9;($t1-$t0) / 1000000000" | bc)
echo $t2 seconds
date

Python 3.10.18
run ingest using beam and DirectRunner
tests done
87.611079706 seconds
Thu Oct  2 11:01:09 PM UTC 2025


In [13]:
%%bash

. /usr/local/bin/activate
conda activate my_tfx_env

python --version

mkdir /kaggle/working/bin
rm -rf /kaggle/working/bin/*

t0=$(date +%s%N)

python -m unittest IngestMovieLensComponentTest.py

t1=$(date +%s%N)
t2=$(echo "scale=9;($t1-$t0) / 1000000000" | bc)
echo $t2 seconds
date

Python 3.10.18
4.880092167 seconds
Thu Oct  2 11:01:16 PM UTC 2025


E
ERROR: IngestMovieLensComponentTest (unittest.loader._FailedTest)
----------------------------------------------------------------------
ImportError: Failed to import test module: IngestMovieLensComponentTest
Traceback (most recent call last):
  File "/usr/local/envs/my_tfx_env/lib/python3.10/unittest/loader.py", line 154, in loadTestsFromName
    module = __import__(module_name)
  File "/kaggle/working/IngestMovieLensComponentTest.py", line 24, in <module>
    from ingest_movie_lens_tfx import IngestMovieLensComponent, IngestMovieLensExecutor
ModuleNotFoundError: No module named 'ingest_movie_lens_tfx'


----------------------------------------------------------------------
Ran 1 test in 0.000s

FAILED (errors=1)
